In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [3]:
seed = 11667
n_collect = 200
cvs = np.concatenate([np.linspace(0, 0.2, 11), np.linspace(0.2, 0.5, 13)[1:]])
coverage0 = 0.2

folders = [
#     'dy_hi', 
#     'dy_lo',
    'dy_ladd'
]

In [4]:
def collect_fux(pars_post, coverage=coverage0, alg="VSC"):    
    m = ModelIntv()
    
    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)

        _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv={}, polished=True)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        
        ms_stats.update({'Key': i, 'Scenario': 'Baseline', 'FollowUp': 0, 'Duration': 0})
        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            for dur in np.linspace(0, 4, 9):
                intv = {'FullACF': {'Coverage': coverage, 'FollowUp': n_fu, 'Duration': dur, 'ScreenAlg': alg}}
                
                _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv=intv, polished=True)
                
                ms = ms.reset_index().assign(Key=i, FollowUp=n_fu, Duration=dur)
                mss.append(ms)

                ms_stats.update({'Key': i, 'Scenario': 'Intervention', 'FollowUp': n_fu, 'Duration': dur})
                mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [ ]:
for folder in folders:
    print(folder)
    for cov in [0.1, 0.2, 0.5]:
        print(folder, cov)
        
        rd.seed(seed)
        
        out_path = f'out/{folder}'
        pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

        sel = rd.choice(list(range(len(pars_post))), n_collect)
        sel.sort()
        pars_post = [pars_post[i] for i in sel]

        mss, mss_stats = collect_fux(pars_post, coverage=cov)
        mss_stats = mss_stats.fillna(0)
        mss.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}.csv')
        mss_stats.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}_stats.csv')

dy_ladd
dy_ladd 0.1
dy_ladd 0.2
dy_ladd 0.5


In [ ]:
mss_stats